# 中文情感分析
Chinese sentiment analysis

### 载入所需的库

In [3]:
import sys
# reload(sys)
# sys.setdefaultencoding('utf8')
import numpy as np 
import pandas as pd
import jieba

from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from sklearn.svm import SVC

from gensim.models.word2vec import Word2Vec


### 加载数据，做预处理（分词），切分训练集和测试集